In [1]:
import pandas as pd
import numpy as np
from processer import preprocessSmilesList, fragmentSmilesList, getSelfiesToks, augSmiles
from multiprocesspandas import applyparallel
from rdkit import Chem

In [2]:
## these smiles data are provided by grisoni github (BIMODAL work)
rawData= pd.read_csv('data/chembl_smiles.tar.xz', 
                     compression="xz", skipfooter= 1, engine='python').astype(str).squeeze()

In [3]:
rawData

0                  CSc1nn(-c2cccc(C)c2)c2cc(C3=CCNCC3)ccc21
1                  CSc1nn(-c2cccc(F)c2)c2cc(C3=CCNCC3)ccc21
2                  CSc1nn(-c2ccc(C)cc2)c2cc(C3=CCNCC3)ccc21
3         O=C(c1ccc2c3c(cnn3C3CCCC3)c(=O)[nH]c2c1)N1CCc2...
4         Cc1cc2[nH]c(=O)c3cnn(C4CCOCC4)c3c2cc1C(=O)N1CC...
                                ...                        
677039    COc1cccc(-c2ccc3c(c2)C2(COC(N)=N2)C2(COC2)C(C)...
677040    CC1(C)Oc2ccc(-c3cc(Cl)cnc3F)cc2C2(COC(N)=N2)C1...
677041       NC1=NC2(c3cc(-c4cncnc4)ccc3OCC23CC3)C(F)(F)CS1
677042    CS(=O)(=O)Nc1ccc(OC[C@@H](O)CN2CCN(c3ccc(Cl)cc...
677043    O[C@H](COc1ccc2[nH]ncc2c1)CN1CCN(c2ccc(Cl)cc2)CC1
Name: chembl_smiles.csv                                                                                   000664  001750  001750  00237257540 13547162144 014727  0                                                                                                    ustar 00robin                           robin                           00000

In [4]:
prepData = preprocessSmilesList(rawData)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 677044/677044 [00:01<00:00, 439302.86it/s]


In [5]:
# so much data are discarded. Optical stereochemical is kept
prepData.shape

(340301,)

In [6]:
pathData = "experiments/data/"

In [7]:
grisoni_fragSmiles = fragmentSmilesList(prepData)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 340301/340301 [00:08<00:00, 39054.63it/s]


In [8]:
grisoni_fragSmiles_10_32 = grisoni_fragSmiles.query('nToks >=10 and nToks<=32')

In [9]:
grisoni_train = grisoni_fragSmiles_10_32.copy()
grisoni_train.rename(columns={'tokens':'fragsmiles'},inplace=True)
grisoni_train.drop(columns='nToks', inplace=True)
grisoni_train['selfies']=grisoni_train['smiles'].apply_parallel(getSelfiesToks, num_processes=30)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 270408/270408 [00:00<00:00, 447856.17it/s]


In [ ]:
grisoni_train

,smiles,fragsmiles,selfies
0,CSc1nn(-c2cccc(C)c2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
1,CSc1nn(-c2cccc(F)c2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
2,CSc1nn(-c2ccc(C)cc2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
3,O=C(c1ccc2c(c1)[nH]c(=O)c1cnn(C3CCCC3)c12)N1CC...,"(Cl, <0>, c1ccc2c(c1)CCN2, <8>, C=O, <5>, O=c1...","([O], [=C], [Branch2], [Ring2], [C], [C], [=C]..."
4,Cc1cc2[nH]c(=O)c3cnn(C4CCOCC4)c3c2cc1C(=O)N1CC...,"(C, <5>, O=c1[nH]c2ccccc2c2[nH]ncc12, <6>, <10...","([C], [C], [=C], [C], [NH1], [C], [=Branch1], ..."
...,...,...,...
677034,OC[C@@H]1CCCN1Cc1nc2ccccc2n1Cc1ccc(Cl)cc1,"(O, C, <4S>, C1CCNC1, <3>, C, <5>, c1ccc2[nH]c...","([O], [C], [C@@H1], [C], [C], [C], [N], [Ring1..."
677035,Cc1ccc2nc(CN3CCCC3)n(Cc3ccc(Cl)cc3)c2c1,"(C1CCNC1, <3>, C, <5>, c1ccc2[nH]cnc2c1, <4>, ...","([C], [C], [=C], [C], [=C], [N], [=C], [Branch..."
677041,NC1=NC2(c3cc(-c4cncnc4)ccc3OCC23CC3)C(F)(F)CS1,"(F, <3>, C1=NC2(CCS1)c1ccccc1OCC21CC1, <0>, <3...","([N], [C], [=N], [C], [Branch2], [Ring1], [N],..."
677042,CS(=O)(=O)Nc1ccc(OC[C@@H](O)CN2CCN(c3ccc(Cl)cc...,"(Cl, <3>, c1ccccc1, <0>, <5>, C1CNCCN1, <2>, C...","([C], [S], [=Branch1], [C], [=O], [=Branch1], ..."


In [10]:
grisoni_fragSmilesAug5 = fragmentSmilesList( grisoni_train["smiles"], augmentation=5 )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1352040/1352040 [00:44<00:00, 30183.49it/s]


In [11]:
grisoni_SmilesAug5 = grisoni_train["smiles"].apply_parallel(augSmiles)
grisoni_SmilesAug5 = grisoni_SmilesAug5.explode()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 270408/270408 [00:00<00:00, 371896.33it/s]


In [12]:
grisoni_trainAug5 = grisoni_SmilesAug5.to_frame(name='smiles')

In [13]:
grisoni_trainAug5['fragsmiles']=grisoni_fragSmilesAug5['tokens']

In [14]:
grisoni_trainAug5['selfies']=grisoni_trainAug5['smiles'].apply_parallel(getSelfiesToks, num_processes=30)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1352040/1352040 [00:03<00:00, 345763.14it/s]


In [17]:
## some molecules cant be represented more than 1/2/3 fragSmiles representation
## specifically 61,870
grisoni_trainAug5[grisoni_trainAug5.isnull().any(axis=1)].index.unique()

Index([     3,      6,      8,    102,    233,    234,    235,    236,    239,
          240,
       ...
       676851, 676852, 676853, 676854, 676912, 676924, 677023, 677025, 677033,
       677034],
      dtype='int64', length=61870)

In [ ]:
from sklearn.model_selection import KFold

In [41]:
def saveSplittedDataset(dataset, suffix=''):

    indexes = dataset.index.unique()

    kf=KFold(n_splits=5, shuffle=True, random_state=0)

    kf = kf.split( indexes )

    for fold,(trainIdx,testIdx) in enumerate(kf):
        train = dataset.loc[indexes[trainIdx]]
        test = dataset.loc[indexes[testIdx]]

        train.to_csv(pathData+f"grisoni_train{suffix}_{fold}.tar.xz", compression="xz", index=False)
        test.to_csv(pathData+f"grisoni_test{suffix}_{fold}.tar.xz", compression="xz", index=False)

In [ ]:
saveSplittedDataset(grisoni_train)

In [ ]:
saveSplittedDataset(grisoni_trainAug5, suffix='Aug5')